In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
%matplotlib inline
%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams
rcParams['savefig.dpi'] = 550
rcParams['font.size'] = 20
plt.rc('font', family='serif')
import lsdb
from tqdm import tqdm
import dask
from dask.distributed import Client
dask.config.set({"temporary-directory" :'/epyc/ssd/users/atzanida/tmp'})
dask.config.set({"dataframe.shuffle-compression": 'Snappy'})

In [10]:
# magic for real time editing of .py files
%load_ext autoreload
%autoreload 2

# Custom functions and tools 
import sys
sys.path.insert(1, '../../dipper')

import tools as my_tools
import dipper as dip_pipeline
import models as dip_models
from evaluate import evaluate as evaluate
from evaluate import evaluate_updated
from gpmcmc import model_gp
from evaluate import half_eval as half_eval

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
%%time
gaia = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/gaia_dr3/gaia", 
                        columns=['ra', 'dec', 'parallax', 'parallax_over_error', 
                                'bp_rp', 'solution_id', 
                                'source_id', 
                                'pmra', 'pmra_error', 
                                'pmdec', 'pmdec_error', 
                                'parallax_error', 
                                'phot_g_mean_mag', 
                                'l', 'b', 'non_single_star', 
                                'classprob_dsc_combmod_galaxy', 
                                'classprob_dsc_combmod_star', 
                                 'in_qso_candidates'])

# load ZTF object table
ztf = lsdb.read_hipscat("/epyc/data3/hipscat/catalogs/ztf_axs/ztf_dr14")

# Load ZTF DR14 sources
ztf_sources = lsdb.read_hipscat("/epyc/data3/hipscat/catalogs/ztf_axs/ztf_source")
# change to ztf_zource

CPU times: user 38.8 s, sys: 6.38 s, total: 45.2 s
Wall time: 45.2 s


In [30]:
from distributed import Client
client = Client(n_workers=20, threads_per_worker=1, memory_limit='25GB')

In [31]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 20
Total threads: 20,Total memory: 465.66 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46139,Workers: 20
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 465.66 GiB
Comm: tcp://127.0.0.1:44988,Total threads: 1
Dashboard: http://127.0.0.1:35477/status,Memory: 23.28 GiB
Nanny: tcp://127.0.0.1:38119,


In [32]:
import dask.dataframe as dd
from tape import Ensemble, ColumnMapper
pd.options.mode.chained_assignment = None

# Initialize an Ensemble
ens = Ensemble(client=client)

In [33]:
ens.client_info()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 20
Total threads: 20,Total memory: 465.66 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46139,Workers: 20
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 465.66 GiB
Comm: tcp://127.0.0.1:44988,Total threads: 1
Dashboard: http://127.0.0.1:35477/status,Memory: 23.28 GiB
Nanny: tcp://127.0.0.1:38119,


In [16]:
%%time
fgk_object = lsdb.read_hipscat("/nvme/users/atzanida/tmp/sample_final_starhorse_hips")

CPU times: user 60.3 ms, sys: 23.1 ms, total: 83.4 ms
Wall time: 60.6 ms


In [17]:
%%time
_sources = fgk_object.join(
    ztf_sources, left_on="ps1_objid_ztf_dr14", right_on="ps1_objid")

CPU times: user 2min 33s, sys: 13 s, total: 2min 46s
Wall time: 2min 40s


In [34]:
# ColumnMapper Establishes which table columns map to timeseries quantities
colmap = ColumnMapper(
        id_col='_hipscat_index',
        time_col='mjd',
        flux_col='mag',
        err_col='magerr',
        band_col='band',
      )

ens.from_dask_dataframe(
    source_frame=_sources._ddf,
    object_frame=fgk_object._ddf,
    column_mapper=colmap,
    sync_tables=False, # Avoid doing an initial sync
    sorted=True, # If the input data is already sorted by the chosen index
    sort=False,
)

In [35]:
# feature evaluation 
column_names = ['Nphot',
    'biweight_scale',
    'frac_above_2_sigma', # in deviation
    'Ndips',
    'rate',
    'chi2dof',
    'skew', 
    'kurtosis',
    'mad',
    'stetson_i',
    'stetson_j',
    'stetson_k',
    'invNeumann',    
    'best_dip_power',
    'best_dip_time_loc',
    'best_dip_start',
    'best_dip_end',
    'best_dip_dt',
    'best_dip_ndet',
    'lc_score']

# Define DataFrame with loc and scale as meta
my_meta = pd.DataFrame(columns=column_names, dtype=float)

In [36]:
%%time
# apply calc_biweight function
calc_ = ens.batch(
    evaluate_updated,
    'mjd_ztf_zource', 'mag_ztf_zource', 
    'magerr_ztf_zource', 'catflags_ztf_zource',
    'band_ztf_zource',
    meta=my_meta,
    use_map=True)

Using generated label, result_1, for a batch result.
CPU times: user 5 ms, sys: 649 µs, total: 5.65 ms
Wall time: 48.7 ms


In [ ]:
%%time
calc_.head(50, npartitions=5)

In [50]:
# previous working example as of April 4th 
%%time
calc_.head(1_000, npartitions=10)

CPU times: user 21 s, sys: 8.35 s, total: 29.4 s
Wall time: 33.2 s


,Nphot,biweight_scale,frac_above_2_sigma,Ndips,rate,chi2dof,skew,kurtosis,mad,stetson_i,stetson_j,stetson_k,invNeumann,best_dip_power,best_dip_time_loc,best_dip_start,best_dip_end,best_dip_dt,best_dip_ndet,lc_score
_hipscat_index,,,,,,,,,,,,,,,,,,,,
213961119105024,269.0,0.015609,0.018587,0.0,0.000000,1.003911,0.289721,0.743892,0.010347,95.148529,64.077812,0.794315,0.692493,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572198062391296,268.0,0.014040,0.018657,0.0,0.000000,1.005126,-0.240236,0.024645,0.008913,23.233597,13.197321,0.779318,0.510219,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471620435050496,217.0,0.013079,0.027650,0.0,0.000000,1.003753,0.198554,2.049891,0.008455,43.642849,35.884365,0.743563,0.602470,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497394970001408,163.0,0.013869,0.000000,0.0,0.000000,1.012015,-4.228230,34.592444,0.008417,111.317398,54.295994,0.606832,0.501397,NaN,NaN,NaN,NaN,NaN,NaN,NaN
778339828629504,216.0,0.012959,0.032407,0.0,0.000000,1.000068,-0.114340,0.443450,0.008190,19.119101,20.018957,0.783205,0.582066,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30574627545677824,247.0,0.012867,0.032389,0.0,0.000000,1.014547,0.613241,1.640722,0.007657,66.591125,40.524063,0.746860,0.775567,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30828687658057728,248.0,0.012120,0.016129,0.0,0.000000,1.004207,-0.329987,0.652126,0.007340,27.404694,21.355305,0.769521,0.558814,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31221926558957568,250.0,0.014558,0.012000,1.0,0.000662,1.000003,0.657509,4.212327,0.009738,36.517708,24.180901,0.755607,0.422658,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
%%time
ens.object.join(calc_).update_ensemble()

# previous working example as of April 4th

In [ ]:
%%time
full_table = ens.object.compute() # full table compute
# previous working example as of April 4th

2024-04-05 17:17:42,886 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/worker.py", line 1252, in heartbeat
    response = await retry_operation(
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/utils_comm.py", line 455, in retry_operation
    return await retry(
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/utils_comm.py", line 434, in retry
    return await cor

2024-04-05 17:46:05,882 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-04-05 17:46:26,026 - distributed.worker.memory - WARNING - gc.collect() took 12.973s. This is usually a sign that some tasks handle too many Python objects at the same time. Rechunking the work into smaller tasks might help.
2024-04-05 17:46:26,027 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 37.27 GiB -- Worker memory limit: 46.57 GiB
2024-04-05 17:46:27,232 - distributed.worker.memory - WARNING - Worker is at 67% memory usage. Resuming worker. Process memory: 31.34 GiB -- Worker memory limit: 46.57 GiB
2024-04-05 17:46:33,069 - distributed.worker.memory - WARNING - Worker is at 90% memory usage. Pausing worker.  Process memory: 41.94 GiB -- Worker memory limit: 46.57 GiB
2024-04-05 17:46:35,516 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold:

2024-04-05 17:46:46,457 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:45341
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/comm/tcp.py", line 230, in read
    buffer = await read_bytes_rw(stream, buffer_nbytes)
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/comm/tcp.py", line 366, in read_bytes_rw
    actual = await stream.read_into(chunk)  # type: ignore[arg-type]
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/worker.py", line 2059, in gather_dep
    response = await get_data_from_worker(
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/worker.py"

2024-04-05 17:48:39,548 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:36932
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/comm/tcp.py", line 230, in read
    buffer = await read_bytes_rw(stream, buffer_nbytes)
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/comm/tcp.py", line 366, in read_bytes_rw
    actual = await stream.read_into(chunk)  # type: ignore[arg-type]
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/worker.py", line 2059, in gather_dep
    response = await get_data_from_worker(
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/worker.py"

2024-04-05 17:49:47,740 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:36932
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/comm/tcp.py", line 546, in connect
    stream = await self.client.connect(
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/tornado/tcpclient.py", line 279, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/asyncio/tasks.py", line 456, in wait_for
    return fut.result()
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_dem

...

2024-04-05 18:13:34,956 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:44539
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/comm/tcp.py", line 297, in write
    raise StreamClosedError()
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/worker.py", line 2059, in gather_dep
    response = await get_data_from_worker(
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/worker.py", line 2863, in get_data_from_worker
    response = await send_recv(
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/core.py", line 1152, in send_recv
    await comm.write(msg, serial

2024-04-05 18:18:21,960 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-04-05 18:18:27,520 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-04-05 18:18:32,078 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-04-05 18:18:32,079 - distributed.worker.memory - WARNING - gc.collect() took 31.539s. This is usually a sign that some tasks handle too many Python objects at the same time. Rechunking the work into smaller tasks might help.
2024-04-05 18:19:14,528 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2024-04-05 18:19:22,396 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
2024-04-05 18:19:22,397 - distributed.worker.memory - WARNING - gc.collect() took 23.698s. This is usually a sign that s

2024-04-05 18:24:50,762 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 30.00 GiB -- Worker memory limit: 46.57 GiB
2024-04-05 18:24:51,248 - distributed.worker.memory - WARNING - Worker is at 70% memory usage. Resuming worker. Process memory: 32.69 GiB -- Worker memory limit: 46.57 GiB
2024-04-05 18:24:53,720 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2024-04-05 18:25:02,205 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-04-05 18:25:02,206 - distributed.worker.memory - WARNING - gc.collect() took 23.753s. This is usually a sign that some tasks handle too many Python objects at the same time. Rechunking the work into smaller t

Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/comm/tcp.py", line 230, in read
    buffer = await read_bytes_rw(stream, buffer_nbytes)
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/comm/tcp.py", line 366, in read_bytes_rw
    actual = await stream.read_into(chunk)  # type: ignore[arg-type]
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/worker.py", line 2059, in gather_dep
    response = await get_data_from_worker(
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/worker.py", line 2863, in get_data_from_worker
    response = await send_recv(
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_

2024-04-05 18:27:06,821 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-04-05 18:27:06,831 - distributed.worker.memory - WARNING - gc.collect() took 21.455s. This is usually a sign that some tasks handle too many Python objects at the same time. Rechunking the work into smaller tasks might help.
2024-04-05 18:27:27,841 - distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
2024-04-05 18:28:11,744 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2024-04-05 18:28:18,511 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-04-05 18:28:39,897 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
2024-04-05 18:28:42,155 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (thresh